<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/CNN_LSTM_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Input, TimeDistributed, LSTM
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Mount Google Drive for dataset access
from google.colab import drive
drive.mount('/content/drive')

# Define dataset paths
dataset_path = '/content/drive/MyDrive/archive1'
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

# Image dimensions and batch size
image_size = (128, 128)  # Adjust to match LSTM architecture input
batch_size = 16

# Data augmentation setup
train_data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for training
train_generator = train_data_gen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True
)

# Preprocessing for testing
test_data_gen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_data_gen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)

# Define CNN-LSTM Model
input_shape = (1, 128, 128, 3)  # TimeDistributed expects 5D input

inputs = Input(shape=input_shape)

# TimeDistributed Conv2D + MaxPooling2D
x = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid'))(inputs)
x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)

x = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='valid'))(x)
x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)

# Flatten within TimeDistributed
x = TimeDistributed(Flatten())(x)

# LSTM layer
x = LSTM(100, activation='tanh')(x)

# Fully connected output layer
outputs = Dense(2, activation='softmax')(x)

# Create Model
model = Model(inputs, outputs)

# Compile the model with binary_crossentropy
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Model Summary
model.summary()

def reshape_data(data_generator):
    for batch_x, batch_y in data_generator:
        # Reshape data to (batch_size, timesteps, height, width, channels)
        # Here, timesteps is set to 1
        batch_x = batch_x.reshape(batch_x.shape[0], 1, *batch_x.shape[1:])
        yield batch_x, batch_y

# Update model training
history = model.fit(
    reshape_data(train_generator),  # Reshape training data
    epochs=25,
    validation_data=reshape_data(test_generator),  # Reshape validation data
    verbose=1
)

# Save the trained model
model.save('/content/drive/MyDrive/cnn_lstm_model_with_augmentation_20epochs.h5')

# Visualize Training Results
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Plot training and validation accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes))

# Final Accuracy
final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
print(f"\nFinal Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy: {final_val_accuracy:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 4861 images belonging to 2 classes.
Found 1603 images belonging to 2 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1, 128, 128, 3)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_10                  │ (None, 1, 126, 126, 64)     │           1,792 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_11                  │ (None, 1, 63, 63, 64)       │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_12                  │ (None, 1, 61, 61, 32)       │          18,464 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_13                  │ (None, 1, 30, 30, 32)       │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_14                  │ (None, 1, 28800)            │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │      11,560,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             202 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,580,858 (44.18 MB)

 Trainable params: 11,580,858 (44.18 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
   4918/Unknown 5418s 1s/step - accuracy: 0.6272 - loss: 0.6388

KeyboardInterrupt: 